In [5]:
import os
import pandas as pd
import numpy as np

def load_clean_triples(file_path, lang_code='en', sample_size=None):
    """
    Load and clean ConceptNet triples for a specific language.
    
    Args:
        file_path (str): Full path to ConceptNet TSV file
        lang_code (str): Language code to filter (e.g. 'en', 'de')
        sample_size (int): Optional max number of triples to return

    Returns:
        List of (start, rel, end) tuples
    """
    df = pd.read_csv(
        file_path, sep='\t', header=None,
        names=['uri', 'rel', 'start', 'end', 'info'], dtype=str
    )
    
    lang_prefix = f'/c/{lang_code}/'
    df = df[
        df['start'].str.startswith(lang_prefix) &
        df['end'].str.startswith(lang_prefix)
    ].copy()

    df['start'] = df['start'].str.split('/').str[3]
    df['rel']   = df['rel'].str.split('/').str[-1]
    df['end']   = df['end'].str.split('/').str[3]

    triples = list(df[['start', 'rel', 'end']].itertuples(index=False, name=None))
    
    if sample_size and sample_size < len(triples):
        import random
        triples = random.sample(triples, sample_size)

    return triples

def extract_vocab(triples):
    """
    Given list of (start, rel, end), return sorted list of unique words.
    """
    vocab = sorted(set([s for s, _, _ in triples] + [e for _, _, e in triples]))
    return vocab

data_path  = r'C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data'
input_data_path = os.path.join(data_path, 'Input')

en_file_name = 'conceptnet-assertions-5.7.0.en.tsv'
en_file_path = os.path.join(input_data_path, en_file_name)
de_file_name = 'conceptnet-assertions-5.7.0.de.tsv'
de_file_path = os.path.join(input_data_path, de_file_name)

# en_triples = load_clean_triples(en_file_path, lang_code='en', sample_size=100_000)
# de_triples = load_clean_triples(de_file_path, lang_code='de', sample_size=100_000)

de_triples = load_clean_triples(de_file_path, lang_code='de')
en_triples = load_clean_triples(en_file_path, lang_code='en')
for lang, triples in [('en', en_triples), ('de', de_triples)]:
    vocab = extract_vocab(triples)
    print(f"Language: {lang}, Vocabulary Size: {len(vocab)}")
    print(f"Sample Vocabulary: {vocab[:10]}")  # Print first 10 words as sample
    print()  # Newline for better readability

Language: en, Vocabulary Size: 1165190
Sample Vocabulary: ['0', '0.22_inch_calibre', "0.3_of_world's_population", '0.4_percent_aluminum', '0.6_percent_calcium', '000', '007', '007s', '00_s', '00s']

Language: de, Vocabulary Size: 519138
Sample Vocabulary: ['0', '00', '0815', '08_15', '1', '1,1,1_trichlor_2,2_bis_4_chlorophenyl_ethan', '1,1_dimethylhydrazin', '1,2,3_propentricarbonsäure', '1,2_benzoldicarbonsäure', '1,2_dichlorethan']



In [8]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict
from collections import defaultdict
import random

class RSCAgent:
    def __init__(self, initial_vocab: List[str]):
        """
        Initialize an agent with an initial vocabulary and relational memory.
        
        Args:
            initial_vocab (List[str]): Starting vocabulary for the agent
        """
        # Relational memory tracks symbol interactions
        self.relational_memory = defaultdict(lambda: defaultdict(int))
        
        # Vocabulary of known symbols
        self.vocab = set(initial_vocab)
        
        # Semantic anchors tracking symbol relationships
        self.semantic_anchors = defaultdict(set)
    
    def evaluate_relation(self, start: str, rel: str, end: str) -> float:
        """
        Evaluate the consistency of a proposed relation.
        
        Args:
            start (str): Starting symbol
            rel (str): Relation type
            end (str): Ending symbol
        
        Returns:
            float: Confidence score for the relation (0-1)
        """
        # If symbols not in vocab, initial skepticism
        if start not in self.vocab or end not in self.vocab:
            return 0.1
        
        # Existing relation count increases confidence
        existing_count = self.relational_memory[start][rel]
        total_relations = sum(self.relational_memory[start].values())
        
        # Compute a confidence score based on existing relations
        if total_relations == 0:
            return 0.5  # Neutral initial stance
        
        confidence = (existing_count / total_relations) * 0.9 + 0.1
        return confidence
    
    def update_relation(self, start: str, rel: str, end: str, accept: bool):
        """
        Update relational memory based on relation validation.
        
        Args:
            start (str): Starting symbol
            rel (str): Relation type
            end (str): Ending symbol
            accept (bool): Whether the relation was accepted
        """
        # Add new symbols to vocabulary if not present
        self.vocab.update([start, end])
        
        if accept:
            # Increment relation count
            self.relational_memory[start][rel] += 1
            
            # Update semantic anchors
            self.semantic_anchors[start].add((rel, end))
            self.semantic_anchors[end].add((rel.replace('Inverse', '') if 'Inverse' in rel else f'{rel}Inverse', start))

def run_rsc_experiment(
    en_triples: List[Tuple[str, str, str]], 
    de_triples: List[Tuple[str, str, str]], 
    num_iterations: int = 1000
) -> Dict:
    """
    Run the Relational Semantic Convergence experiment.
    
    Args:
        en_triples (List[Tuple[str, str, str]]): English language triples
        de_triples (List[Tuple[str, str, str]]): German language triples
        num_iterations (int): Number of interaction iterations
    
    Returns:
        Dict: Experiment results and metrics
    """
    # Initial vocabularies
    en_vocab = list(set([s for s, _, _ in en_triples] + [e for _, _, e in en_triples]))
    de_vocab = list(set([s for s, _, _ in de_triples] + [e for _, _, e in de_triples]))
    
    # Create agents
    en_agent = RSCAgent(en_vocab)
    de_agent = RSCAgent(de_vocab)
    
    # Tracking convergence metrics
    convergence_metrics = {
        'shared_vocab_size': 0,
        'shared_relations': 0,
        'semantic_anchor_overlap': 0
    }
    
    # Experiment iterations
    for _ in range(num_iterations):
        # Randomly select triples from each language
        en_triple = random.choice(en_triples)
        de_triple = random.choice(de_triples)
        
        # Evaluate and potentially accept relations
        en_confidence = en_agent.evaluate_relation(*en_triple)
        de_confidence = de_agent.evaluate_relation(*de_triple)
        
        # Simple probabilistic acceptance based on confidence
        en_accept = random.random() < en_confidence
        de_accept = random.random() < de_confidence
        
        # Update agents' relational memories
        en_agent.update_relation(*en_triple, en_accept)
        de_agent.update_relation(*de_triple, de_accept)
        
        # Compute convergence metrics
        convergence_metrics['shared_vocab_size'] = len(en_agent.vocab.intersection(de_agent.vocab))
        
        # Compute shared relations
        shared_relations = 0
        for start in en_agent.semantic_anchors:
            if start in de_agent.semantic_anchors:
                shared_relations += len(en_agent.semantic_anchors[start].intersection(de_agent.semantic_anchors[start]))
        convergence_metrics['shared_relations'] = shared_relations
    
    return convergence_metrics

# Experiment execution (commented out for now)
results = run_rsc_experiment(en_triples, de_triples)
print("Experiment Results:", results)

Experiment Results: {'shared_vocab_size': 26284, 'shared_relations': 0, 'semantic_anchor_overlap': 0}


# Version 2

In [13]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict
from collections import defaultdict
import random
from tqdm import tqdm

class RSCAgent:
    def __init__(self, initial_vocab: List[str]):
        """
        Initialize an agent with an initial vocabulary and relational memory.
        
        Args:
            initial_vocab (List[str]): Starting vocabulary for the agent
        """
        # Relational memory tracks symbol interactions
        self.relational_memory = defaultdict(lambda: defaultdict(int))
        
        # Vocabulary of known symbols
        self.vocab = set(initial_vocab)
        
        # Semantic anchors tracking symbol relationships
        self.semantic_anchors = defaultdict(set)
    
    def evaluate_relation(self, start: str, rel: str, end: str) -> float:
        """
        Enhanced relation evaluation with more flexible confidence scoring.
        
        Args:
            start (str): Starting symbol
            rel (str): Relation type
            end (str): Ending symbol
        
        Returns:
            float: Confidence score for the relation (0-1)
        """
        # If symbols not in vocab, moderate initial openness
        if start not in self.vocab or end not in self.vocab:
            return 0.4  # More open to new relations
        
        # Existing relation count increases confidence
        existing_count = self.relational_memory[start][rel]
        total_relations = sum(self.relational_memory[start].values())
        
        # More nuanced confidence calculation
        if total_relations == 0:
            return 0.6  # More optimistic initial stance
        
        # Exponential decay to prevent over-fitting to existing relations
        confidence = min(
            0.9,  # Cap confidence
            (existing_count / total_relations) ** 0.7 * 0.8 + 0.2
        )
        
        return confidence

    def find_potential_translation(self, word: str, other_vocab: set) -> str:
        """
        Attempt to find potential translation using similarity heuristics.
        
        Args:
            word (str): Source word
            other_vocab (set): Vocabulary of target language
        
        Returns:
            str: Potential translation or original word
        """
        # Simple heuristics for finding potential translations
        # 1. Exact match
        if word in other_vocab:
            return word
        
        # 2. Substring match (very naive, for demonstration)
        for candidate in other_vocab:
            if word.lower() in candidate.lower() or candidate.lower() in word.lower():
                return candidate
        
        return word  # Fallback to original word
    
    def update_relation(self, start: str, rel: str, end: str, accept: bool):
        """
        Update relational memory based on relation validation.
        
        Args:
            start (str): Starting symbol
            rel (str): Relation type
            end (str): Ending symbol
            accept (bool): Whether the relation was accepted
        """
        # Add new symbols to vocabulary if not present
        self.vocab.update([start, end])
        
        if accept:
            # Increment relation count
            self.relational_memory[start][rel] += 1
            
            # Update semantic anchors
            self.semantic_anchors[start].add((rel, end))
            self.semantic_anchors[end].add((rel.replace('Inverse', '') if 'Inverse' in rel else f'{rel}Inverse', start))

def run_rsc_experiment(
    en_triples: List[Tuple[str, str, str]], 
    de_triples: List[Tuple[str, str, str]], 
    num_iterations: int = 10000  # Increased iterations
) -> Dict:
    """
    Enhanced Relational Semantic Convergence experiment.
    
    Args:
        en_triples (List[Tuple[str, str, str]]): English language triples
        de_triples (List[Tuple[str, str, str]]): German language triples
        num_iterations (int): Number of interaction iterations
    
    Returns:
        Dict: Experiment results and metrics
    """
    # Initial vocabularies
    en_vocab = list(set([s for s, _, _ in en_triples] + [e for _, _, e in en_triples]))
    de_vocab = list(set([s for s, _, _ in de_triples] + [e for _, _, e in de_triples]))
    
    # Create agents
    en_agent = RSCAgent(en_vocab)
    de_agent = RSCAgent(de_vocab)
    
    # Tracking convergence metrics
    convergence_metrics = {
        'shared_vocab_size': 0,
        'shared_relations': 0,
        'semantic_anchor_overlap': 0,
        'cross_lingual_relations': 0
    }
    
    # Cross-lingual translation mapping
    de_vocab_set = set(de_vocab)
    
    # Experiment iterations
    for _ in tqdm(range(num_iterations), desc="Running RSC Experiment"):
        # Cross-language interaction
        en_triple = random.choice(en_triples)
        de_triple = random.choice(de_triples)
        
        # Enhanced translation attempt
        en_start_translation = en_agent.find_potential_translation(en_triple[0], de_vocab_set)
        en_end_translation = en_agent.find_potential_translation(en_triple[2], de_vocab_set)
        
        # Evaluate and potentially accept relations
        en_confidence = en_agent.evaluate_relation(*en_triple)
        de_confidence = de_agent.evaluate_relation(*de_triple)
        
        # Cross-lingual relation mapping
        cross_lingual_triple = (
            en_start_translation, 
            en_triple[1],  # Assume relation type is language-independent 
            en_end_translation
        )
        
        # Probabilistic acceptance with enhanced confidence
        en_accept = random.random() < (en_confidence * 0.9 + 0.1)
        de_accept = random.random() < (de_confidence * 0.9 + 0.1)
        
        # Update agents' relational memories
        en_agent.update_relation(*en_triple, en_accept)
        de_agent.update_relation(*de_triple, de_accept)
        
        # Cross-lingual relation exploration
        cross_confidence = (en_confidence + de_confidence) / 2
        cross_accept = random.random() < cross_confidence
        if cross_accept:
            de_agent.update_relation(*cross_lingual_triple, True)
            convergence_metrics['cross_lingual_relations'] += 1
        
        # Compute convergence metrics
        convergence_metrics['shared_vocab_size'] = len(en_agent.vocab.intersection(de_agent.vocab))
        
        # Compute shared relations
        shared_relations = 0
        semantic_anchor_overlap = 0
        for start in en_agent.semantic_anchors:
            if start in de_agent.semantic_anchors:
                relation_intersection = en_agent.semantic_anchors[start].intersection(de_agent.semantic_anchors[start])
                shared_relations += len(relation_intersection)
                semantic_anchor_overlap += len(relation_intersection)
        
        convergence_metrics['shared_relations'] = shared_relations
        convergence_metrics['semantic_anchor_overlap'] = semantic_anchor_overlap
        if _ % 100 == 0:
            # Print progress every 100 iterations
            tqdm.write(f"Iteration {_+1}/{num_iterations} - Shared Vocab: {convergence_metrics['shared_vocab_size']}, "
                       f"Shared Relations: {convergence_metrics['shared_relations']}, "
                       f"Semantic Anchor Overlap: {convergence_metrics['semantic_anchor_overlap']}, "
                       f"Cross-Lingual Relations: {convergence_metrics['cross_lingual_relations']}")
            
    return convergence_metrics

# Experiment execution (commented out for now)
results = run_rsc_experiment(en_triples, de_triples)

Running RSC Experiment:   0%|          | 5/10000 [00:00<07:08, 23.35it/s]

Iteration 1/10000 - Shared Vocab: 26284, Shared Relations: 0, Semantic Anchor Overlap: 0, Cross-Lingual Relations: 0


Running RSC Experiment:   1%|          | 105/10000 [00:05<07:08, 23.07it/s]

Iteration 101/10000 - Shared Vocab: 26284, Shared Relations: 4, Semantic Anchor Overlap: 4, Cross-Lingual Relations: 64


Running RSC Experiment:   2%|▏         | 205/10000 [00:10<07:57, 20.52it/s]

...

Running RSC Experiment:   3%|▎         | 304/10000 [00:14<07:40, 21.04it/s]

Iteration 301/10000 - Shared Vocab: 26284, Shared Relations: 8, Semantic Anchor Overlap: 8, Cross-Lingual Relations: 182


Running RSC Experiment:   4%|▎         | 354/10000 [00:17<07:46, 20.69it/s]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "C:\Users\erich\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\erich\AppData\Local\Temp\ipykernel_24872\3974507376.py", line 196, in <module>
    results = run_rsc_experiment(en_triples, de_triples)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\erich\AppData\Local\Temp\ipykernel_24872\3974507376.py", line -1, in run_rsc_experiment
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\erich\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\erich\AppData\Roaming\Python\Python311\site-packages\IPython\core\ultratb.py", line 1428, in

In [14]:
print("Experiment Results:", results)

Experiment Results: {'shared_vocab_size': 26284, 'shared_relations': 0, 'semantic_anchor_overlap': 0}


# Optmized version

In [ ]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Set
from collections import defaultdict
import random
import multiprocessing
import time

class OptimizedRSCAgent:
    __slots__ = ['relational_memory', 'vocab', 'semantic_anchors']
    
    def __init__(self, initial_vocab: List[str]):
        """
        Optimized initialization with memory-efficient data structures.
        
        Args:
            initial_vocab (List[str]): Starting vocabulary for the agent
        """
        # Use dict instead of defaultdict for slight performance boost
        self.relational_memory = {}
        
        # Use a set for constant-time lookup
        self.vocab = set(initial_vocab)
        
        # Use a dict of frozensets for hashability and memory efficiency
        self.semantic_anchors = {}
    
    def evaluate_relation(self, start: str, rel: str, end: str) -> float:
        """
        Optimized relation evaluation with faster calculations.
        
        Args:
            start (str): Starting symbol
            rel (str): Relation type
            end (str): Ending symbol
        
        Returns:
            float: Confidence score for the relation (0-1)
        """
        # Quick vocab check
        if start not in self.vocab or end not in self.vocab:
            return 0.4
        
        # Faster relation counting
        start_relations = self.relational_memory.get(start, {})
        existing_count = start_relations.get(rel, 0)
        total_relations = sum(start_relations.values()) if start_relations else 0
        
        # Quick confidence calculation
        if total_relations == 0:
            return 0.6
        
        # Streamlined confidence computation
        confidence = min(
            0.9,
            (existing_count / total_relations) ** 0.7 * 0.8 + 0.2
        )
        
        return confidence

    def find_potential_translation(self, word: str, other_vocab: Set[str]) -> str:
        """
        Fast translation lookup with optimized matching.
        
        Args:
            word (str): Source word
            other_vocab (Set[str]): Vocabulary of target language
        
        Returns:
            str: Potential translation or original word
        """
        # Exact match first (fastest)
        if word in other_vocab:
            return word
        
        # Faster substring matching
        word_lower = word.lower()
        for candidate in other_vocab:
            candidate_lower = candidate.lower()
            if (word_lower in candidate_lower or 
                candidate_lower in word_lower):
                return candidate
        
        return word
    
    def update_relation(self, start: str, rel: str, end: str, accept: bool):
        """
        Optimized relation memory update.
        
        Args:
            start (str): Starting symbol
            rel (str): Relation type
            end (str): Ending symbol
            accept (bool): Whether the relation was accepted
        """
        # Efficient vocab update
        self.vocab.update([start, end])
        
        if accept:
            # Efficient relation tracking
            if start not in self.relational_memory:
                self.relational_memory[start] = {}
            self.relational_memory[start][rel] = self.relational_memory[start].get(rel, 0) + 1
            
            # Efficient semantic anchor update
            if start not in self.semantic_anchors:
                self.semantic_anchors[start] = set()
            self.semantic_anchors[start].add((rel, end))
            
            if end not in self.semantic_anchors:
                self.semantic_anchors[end] = set()
            inverse_rel = rel.replace('Inverse', '') if 'Inverse' in rel else f'{rel}Inverse'
            self.semantic_anchors[end].add((inverse_rel, start))

def run_rsc_experiment(
    en_triples: List[Tuple[str, str, str]], 
    de_triples: List[Tuple[str, str, str]], 
    num_iterations: int = 10000
) -> Dict:
    """
    Highly optimized Relational Semantic Convergence experiment.
    
    Args:
        en_triples (List[Tuple[str, str, str]]): English language triples
        de_triples (List[Tuple[str, str, str]]): German language triples
        num_iterations (int): Number of interaction iterations
    
    Returns:
        Dict: Experiment results and metrics
    """
    # Precompute vocabularies
    en_vocab = list(set([s for s, _, _ in en_triples] + [e for _, _, e in en_triples]))
    de_vocab = list(set([s for s, _, _ in de_triples] + [e for _, _, e in de_triples]))
    de_vocab_set = set(de_vocab)
    
    # Create agents
    en_agent = OptimizedRSCAgent(en_vocab)
    de_agent = OptimizedRSCAgent(de_vocab)
    
    # Preallocate metrics dictionary
    convergence_metrics = {
        'shared_vocab_size': 0,
        'shared_relations': 0,
        'semantic_anchor_overlap': 0,
        'cross_lingual_relations': 0
    }
    
    # Performance optimization: Precompute random choices
    en_random_triples = [random.choice(en_triples) for _ in range(num_iterations)]
    de_random_triples = [random.choice(de_triples) for _ in range(num_iterations)]
    
    # Main experiment loop
    for i in range(num_iterations):
        # Retrieve precomputed random triples
        en_triple = en_random_triples[i]
        de_triple = de_random_triples[i]
        
        # Efficient translation attempt
        en_start_translation = en_agent.find_potential_translation(en_triple[0], de_vocab_set)
        en_end_translation = en_agent.find_potential_translation(en_triple[2], de_vocab_set)
        
        # Evaluate relations
        en_confidence = en_agent.evaluate_relation(*en_triple)
        de_confidence = de_agent.evaluate_relation(*de_triple)
        
        # Probabilistic acceptance
        en_accept = random.random() < (en_confidence * 0.9 + 0.1)
        de_accept = random.random() < (de_confidence * 0.9 + 0.1)
        
        # Update agents
        en_agent.update_relation(*en_triple, en_accept)
        de_agent.update_relation(*de_triple, de_accept)
        
        # Cross-lingual relation exploration
        cross_confidence = (en_confidence + de_confidence) / 2
        cross_accept = random.random() < cross_confidence
        if cross_accept:
            cross_lingual_triple = (
                en_start_translation, 
                en_triple[1],  # Language-independent relation 
                en_end_translation
            )
            de_agent.update_relation(*cross_lingual_triple, True)
            convergence_metrics['cross_lingual_relations'] += 1
        
        # Compute convergence metrics periodically
        if i % 100 == 0:
            # Shared vocabulary
            convergence_metrics['shared_vocab_size'] = len(en_agent.vocab.intersection(de_agent.vocab))
            
            # Shared relations computation
            shared_relations = 0
            semantic_anchor_overlap = 0
            for start in en_agent.semantic_anchors:
                if start in de_agent.semantic_anchors:
                    relation_intersection = en_agent.semantic_anchors[start].intersection(de_agent.semantic_anchors[start])
                    shared_relations += len(relation_intersection)
                    semantic_anchor_overlap += len(relation_intersection)
            
            convergence_metrics['shared_relations'] = shared_relations
            convergence_metrics['semantic_anchor_overlap'] = semantic_anchor_overlap
            
            # Optional: Print progress
            print(f"Iteration {i+1}/{num_iterations} - "
                  f"Shared Vocab: {convergence_metrics['shared_vocab_size']}, "
                  f"Shared Relations: {convergence_metrics['shared_relations']}, "
                  f"Semantic Anchor Overlap: {convergence_metrics['semantic_anchor_overlap']}, "
                  f"Cross-Lingual Relations: {convergence_metrics['cross_lingual_relations']}")
    
    return convergence_metrics

# Performance timing wrapper
def time_experiment(en_triples, de_triples, num_iterations=10000):
    """
    Time the RSC experiment and provide performance metrics.
    
    Args:
        en_triples (List[Tuple]): English language triples
        de_triples (List[Tuple]): German language triples
        num_iterations (int): Number of iterations
    
    Returns:
        Tuple of (experiment results, execution time)
    """
    start_time = time.time()
    results = run_rsc_experiment(en_triples, de_triples, num_iterations)
    end_time = time.time()
    
    print(f"\nExperiment Performance:")
    print(f"Total Execution Time: {end_time - start_time:.2f} seconds")
    print(f"Iterations per Second: {num_iterations / (end_time - start_time):.2f}")
    
    return results, end_time - start_time

# Optional: Parallel experiment execution
def parallel_experiment_runner(en_triples, de_triples, num_iterations=10000, num_runs=3):
    """
    Run multiple experiments in parallel to assess consistency.
    
    Args:
        en_triples (List[Tuple]): English language triples
        de_triples (List[Tuple]): German language triples
        num_iterations (int): Number of iterations per experiment
        num_runs (int): Number of parallel experiment runs
    
    Returns:
        List of experiment results
    """
    with multiprocessing.Pool(processes=max(1, multiprocessing.cpu_count() - 1)) as pool:
        # Create arguments for each experiment run
        experiment_args = [(en_triples, de_triples, num_iterations) for _ in range(num_runs)]
        
        # Run experiments in parallel
        results = pool.starmap(run_rsc_experiment, experiment_args)
    
    return results

# Example usage (uncomment when ready)
results = time_experiment(en_triples, de_triples)
parallel_results = parallel_experiment_runner(en_triples, de_triples)

Iteration 1/10000 - Shared Vocab: 26284, Shared Relations: 0, Semantic Anchor Overlap: 0, Cross-Lingual Relations: 1
Iteration 101/10000 - Shared Vocab: 26284, Shared Relations: 0, Semantic Anchor Overlap: 0, Cross-Lingual Relations: 61
Iteration 201/10000 - Shared Vocab: 26284, Shared Relations: 0, Semantic Anchor Overlap: 0, Cross-Lingual Relations: 121
Iteration 301/10000 - Shared Vocab: 26284, Shared Relations: 2, Semantic Anchor Overlap: 2, Cross-Lingual Relations: 180
Iteration 401/10000 - Shared Vocab: 26284, Shared Relations: 4, Semantic Anchor Overlap: 4, Cross-Lingual Relations: 239
Iteration 501/10000 - Shared Vocab: 26284, Shared Relations: 8, Semantic Anchor Overlap: 8, Cross-Lingual Relations: 299
Iteration 601/10000 - Shared Vocab: 26284, Shared Relations: 14, Semantic Anchor Overlap: 14, Cross-Lingual Relations: 355
Iteration 701/10000 - Shared Vocab: 26284, Shared Relations: 14, Semantic Anchor Overlap: 14, Cross-Lingual Relations: 416
Iteration 801/10000 - Shared Voca